# Importing libraries

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
import re
import nltk
from collections import defaultdict


In [2]:
tokenizer = RegexpTokenizer(r'\w+')


# Importing data

In [3]:
dataset = pd.read_excel('integrated.xlsx')
dataset

,username,date,product,rating,feedback_auto,feedback
0,paje.ph,2022-02-17 16:19:00,Aquaspeed Aquarium Heater,1.0,Damaged / Defective item,DEFFECTIVE ITEM/ SELLER NOT PROFESSIONAL\nWron...
1,kyliegriego,2022-02-11 09:05:00,Airpump Connector,1.0,NaN,Walang dumating na gantong item
2,salvatierraaldrin,2022-02-09 13:08:00,Co2 Diffuser Set,1.0,Damaged / Defective item,NaN
3,salvatierraaldrin,2022-02-09 13:08:00,Aquarium Acrylic Holder,1.0,NaN,NaN
4,salvatierraaldrin,2022-02-09 13:08:00,Spirulina Tablet 50pcs,1.0,NaN,.wlang nalabas n CO2... buti pa yung ginawa ko...
...,...,...,...,...,...,...
5012,crimsonqn,2021-11-08 17:26:00,Sunsun Cleaning Set,5.0,Well-packaged,It’s been my second time ordering at this shop...
5013,neiljohnmanansala,2021-12-14 10:22:00,Sunsun Cleaning Set,5.0,Excellent quality,NaN
5014,totskie17,2021-11-07 10:25:00,Sunsun Cleaning Set,5.0,Excellent quality,NaN
5015,hsiehchung18,2021-10-11 20:03:00,Sunsun Cleaning Set,5.0,Well-packaged,NaN


# Cleaning


In [4]:
dataset.info

<bound method DataFrame.info of                username                date                    product  \
0               paje.ph 2022-02-17 16:19:00  Aquaspeed Aquarium Heater   
1           kyliegriego 2022-02-11 09:05:00          Airpump Connector   
2     salvatierraaldrin 2022-02-09 13:08:00           Co2 Diffuser Set   
3     salvatierraaldrin 2022-02-09 13:08:00    Aquarium Acrylic Holder   
4     salvatierraaldrin 2022-02-09 13:08:00     Spirulina Tablet 50pcs   
...                 ...                 ...                        ...   
5012          crimsonqn 2021-11-08 17:26:00        Sunsun Cleaning Set   
5013  neiljohnmanansala 2021-12-14 10:22:00        Sunsun Cleaning Set   
5014          totskie17 2021-11-07 10:25:00        Sunsun Cleaning Set   
5015       hsiehchung18 2021-10-11 20:03:00        Sunsun Cleaning Set   
5016            s*****4 2021-09-03 12:49:00        Sunsun Cleaning Set   

      rating             feedback_auto  \
0        1.0  Damaged / Defective ite

In [5]:
dataset.isnull().sum()

username            3
date                3
product             2
rating              2
feedback_auto    2511
feedback         2071
dtype: int64

In [6]:
dataset['feedback'] = dataset['feedback'].fillna(0)

for i in range(len(dataset)):    
    if dataset.loc[i,'feedback'] == 0:
        dataset.loc[i,'feedback'] = dataset.loc[i,'feedback_auto']

In [7]:
dataset.isnull().sum()

username            3
date                3
product             2
rating              2
feedback_auto    2511
feedback          817
dtype: int64

In [8]:
dataset.dropna(subset = ["feedback"], inplace =True )
dataset.dropna(subset = ["username"], inplace =True )
dataset.dropna(subset = ["date"], inplace =True )

In [9]:
dataset.isna().sum()


username            0
date                0
product             0
rating              0
feedback_auto    1694
feedback            0
dtype: int64

In [10]:
dataset.dropna(subset = ["username"], inplace =True )
dataset.dropna(subset = ["date"], inplace =True )

In [11]:
dataset.duplicated().sum()

136

In [12]:
dataset = dataset.drop_duplicates()

In [13]:
dataset['date'] = dataset['date'].dt.strftime('%m/%d/%Y')

C:\Users\R\AppData\Local\Temp\ipykernel_12384\2107347625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['date'] = dataset['date'].dt.strftime('%m/%d/%Y')


In [14]:
del dataset['feedback_auto']

In [15]:
del dataset['username']

In [16]:
dataset['rating'] = dataset['rating'].astype(int)
dataset

C:\Users\R\AppData\Local\Temp\ipykernel_12384\2269774767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['rating'] = dataset['rating'].astype(int)


,date,product,rating,feedback
0,02/17/2022,Aquaspeed Aquarium Heater,1,DEFFECTIVE ITEM/ SELLER NOT PROFESSIONAL\nWron...
1,02/11/2022,Airpump Connector,1,Walang dumating na gantong item
2,02/09/2022,Co2 Diffuser Set,1,Damaged / Defective item
4,02/09/2022,Spirulina Tablet 50pcs,1,.wlang nalabas n CO2... buti pa yung ginawa ko...
5,02/07/2022,Aquaspeed Aquarium Heater,1,Basag poyung dumating tsaka aquaspeed po order...
...,...,...,...,...
5011,04/14/2021,Sunsun Cleaning Set,5,All goods item Well packed Mabilis mag ship ka...
5012,11/08/2021,Sunsun Cleaning Set,5,It’s been my second time ordering at this shop...
5013,12/14/2021,Sunsun Cleaning Set,5,Excellent quality
5014,11/07/2021,Sunsun Cleaning Set,5,Excellent quality


# NLP


In [17]:
taglish_stopwords = ["akin","aking","ako","alin","am","amin","aming","ang","ano","anumang","apat","at","atin","ating","ay","bababa","bago","bakit","bawat","bilang","dahil","dalawa","dun","dapat","din","dito","doon","gagawin","gayunman","ginagawa","ginawa","ginawang","gumawa","gusto","habang","hanggang","huwag","iba","ibaba","ibabaw","ibig","ikaw","ilagay","ilalim","ilan","inyong","isa","isang","itaas","ito","iyo","iyon","iyong","ka","kahit","kailangan","kailanman","kami","kanila","kanilang","kanino","kanya","kanyang","kapag","kapwa","karamihan","katiyakan","katulad","kaya","kaysa","ko","kong","kumuha","kung","laban","lahat","lamang","likod","lima","maaari","maaaring","maging","makita","marapat","masyado","may", "ma","mayroon","mga","minsan","mismo","po","mula", "naten","tapos", "basta","sya","mo","nyo","muli","na","nabanggit","naging","nagkaroon","nais","nakita","naka","namin","napaka","narito","nasaan","ng","ngayon","ni","nila","nilang","nito","niya","niyang","noon","o","pa","paano","pababa","paggawa","pagitan","pagkakaroon","pagkatapos","palabas","pamamagitan","panahon", "pag","yan","sayo","nka","nyang","pangalawa","para","paraan","pareho","pataas","pero","pumunta","pumupunta","rin","kya","gnun","sa","saan","sabi","sabihin","sarili","sila","sino","siya","tatlo","talaga","tagalang","tlga","tlgang", "nitong","tayo","tulad","tungkol", "tsaka", "nasa", "daw","una", "nlng", "lng", "me", "naman","nmn", "namang", "si", "man","nman", "mag","mejo", "medyo","yung","ung", "nalang","nd","i","nag","mm", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "also","below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "nor", "own", "same", "so", "than", "too", "very",  "can", "will", "just", "don", "should", "now","a","b","c","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]


In [18]:
tokenizer = RegexpTokenizer(r'\w+')


In [19]:
def getCleanText(text):

    #converting to small letters
    text = text.lower()

    #remove numbers
    text = re.sub(r'[0-9]+','',text)
   
    #tokenize
    tokens = tokenizer.tokenize(text)
   
    #stopwords
    new_tokens = [token for token in tokens if token not in taglish_stopwords]
    
    clean_text = " ".join(new_tokens)
    return clean_text

In [20]:
clean_data = [getCleanText(i) for i in dataset['feedback']]
clean_data

['deffective item seller not professional wrong move dami palang bad reviews shop nato diko nabasa lesson learned review muna shop item buyyout',
 'walang dumating gantong item',
 'damaged defective item',
 'wlang nalabas co buti diy diffuser',
 'basag poyung dumating aquaspeed order ibang brand dumating',
 'pangit nung parts nadumating sira',
 'kulang dinidikit',
 'tagal ship dumating',
 'ordered pcs dumating chatted seller yesterday send stone crack immedietly held see vid seller doesn response yet incident got dissapointed',
 'scam inorder inch quantity dumating sakin inch lang',
 'uses ayaw gumana',
 'bat powder binigay',
 'not receive item',
 'damaged defective item',
 'disappointed hindi dumating white sand worth only received dragon stone imagine almost half binayaran hindi dumating scam',
 'damaged defective item',
 'di dumidikit',
 'tagal dumating blame courier seller nagpapasko requested refund tagal seller di lang update',
 'delayed delivery',
 'walang kwentang seller walang

In [21]:
dataset['feedback'] = clean_data


C:\Users\R\AppData\Local\Temp\ipykernel_12384\3444145888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['feedback'] = clean_data


In [22]:
feedback_data = []
for i in dataset['feedback']:
    feedback_data.append(i.lstrip().rstrip())
 
dataset['feedback'] = feedback_data

    

C:\Users\R\AppData\Local\Temp\ipykernel_12384\3347356108.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['feedback'] = feedback_data


In [23]:
dataset['feedback'] = dataset['feedback'].fillna(' ')


C:\Users\R\AppData\Local\Temp\ipykernel_12384\190073588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['feedback'] = dataset['feedback'].fillna(' ')


# Transforming ratings to labels

In [24]:
dataset

,date,product,rating,feedback
0,02/17/2022,Aquaspeed Aquarium Heater,1,deffective item seller not professional wrong ...
1,02/11/2022,Airpump Connector,1,walang dumating gantong item
2,02/09/2022,Co2 Diffuser Set,1,damaged defective item
4,02/09/2022,Spirulina Tablet 50pcs,1,wlang nalabas co buti diy diffuser
5,02/07/2022,Aquaspeed Aquarium Heater,1,basag poyung dumating aquaspeed order ibang br...
...,...,...,...,...
5011,04/14/2021,Sunsun Cleaning Set,5,goods item well packed mabilis ship dumating a...
5012,11/08/2021,Sunsun Cleaning Set,5,second time ordering shop packed well items no...
5013,12/14/2021,Sunsun Cleaning Set,5,excellent quality
5014,11/07/2021,Sunsun Cleaning Set,5,excellent quality


In [25]:
feedback_label = []
for i in dataset['rating']:
    if i > 3:
        feedback_label.append('Positive')
    elif i<3:
        feedback_label.append('Negative')
    else:
        feedback_label.append('Neutral')


    
dataset['sentiment'] = feedback_label


C:\Users\R\AppData\Local\Temp\ipykernel_12384\2280950592.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['sentiment'] = feedback_label


In [26]:
dataset


,date,product,rating,feedback,sentiment
0,02/17/2022,Aquaspeed Aquarium Heater,1,deffective item seller not professional wrong ...,Negative
1,02/11/2022,Airpump Connector,1,walang dumating gantong item,Negative
2,02/09/2022,Co2 Diffuser Set,1,damaged defective item,Negative
4,02/09/2022,Spirulina Tablet 50pcs,1,wlang nalabas co buti diy diffuser,Negative
5,02/07/2022,Aquaspeed Aquarium Heater,1,basag poyung dumating aquaspeed order ibang br...,Negative
...,...,...,...,...,...
5011,04/14/2021,Sunsun Cleaning Set,5,goods item well packed mabilis ship dumating a...,Positive
5012,11/08/2021,Sunsun Cleaning Set,5,second time ordering shop packed well items no...,Positive
5013,12/14/2021,Sunsun Cleaning Set,5,excellent quality,Positive
5014,11/07/2021,Sunsun Cleaning Set,5,excellent quality,Positive


In [27]:
product_data=[]

for i in dataset['product']:
    product_data.append(i.lstrip().rstrip())
    
dataset['product'] = product_data

C:\Users\R\AppData\Local\Temp\ipykernel_12384\2525402348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['product'] = product_data


# Schema

In [28]:
dataset.insert(0, 'feedback_id', range(1, 1 + len(dataset)))
dataset



,feedback_id,date,product,rating,feedback,sentiment
0,1,02/17/2022,Aquaspeed Aquarium Heater,1,deffective item seller not professional wrong ...,Negative
1,2,02/11/2022,Airpump Connector,1,walang dumating gantong item,Negative
2,3,02/09/2022,Co2 Diffuser Set,1,damaged defective item,Negative
4,4,02/09/2022,Spirulina Tablet 50pcs,1,wlang nalabas co buti diy diffuser,Negative
5,5,02/07/2022,Aquaspeed Aquarium Heater,1,basag poyung dumating aquaspeed order ibang br...,Negative
...,...,...,...,...,...,...
5011,4057,04/14/2021,Sunsun Cleaning Set,5,goods item well packed mabilis ship dumating a...,Positive
5012,4058,11/08/2021,Sunsun Cleaning Set,5,second time ordering shop packed well items no...,Positive
5013,4059,12/14/2021,Sunsun Cleaning Set,5,excellent quality,Positive
5014,4060,11/07/2021,Sunsun Cleaning Set,5,excellent quality,Positive


# Transforming Date Table

In [29]:
date_dataset =pd.DataFrame(dataset['date'])
date_dataset

,date
0,02/17/2022
1,02/11/2022
2,02/09/2022
4,02/09/2022
5,02/07/2022
...,...
5011,04/14/2021
5012,11/08/2021
5013,12/14/2021
5014,11/07/2021


In [30]:
date_dataset = date_dataset.drop_duplicates()

date_dataset['date']=pd.to_datetime(date_dataset['date'])

date_dataset['week']=date_dataset['date'].dt.strftime('%A')
date_dataset['month']=date_dataset['date'].dt.strftime('%b')
date_dataset['year']=date_dataset['date'].dt.strftime('%Y')
date_dataset['day']=date_dataset['date'].dt.strftime('%d')



C:\Users\R\AppData\Local\Temp\ipykernel_12384\1363881000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dataset['date']=pd.to_datetime(date_dataset['date'])
C:\Users\R\AppData\Local\Temp\ipykernel_12384\1363881000.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dataset['week']=date_dataset['date'].dt.strftime('%A')
C:\Users\R\AppData\Local\Temp\ipykernel_12384\1363881000.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [31]:
date_dataset.insert(0, 'date_id', range(1, 1 + len(date_dataset)))
date_dataset


,date_id,date,week,month,year,day
0,1,2022-02-17,Thursday,Feb,2022,17
1,2,2022-02-11,Friday,Feb,2022,11
2,3,2022-02-09,Wednesday,Feb,2022,09
5,4,2022-02-07,Monday,Feb,2022,07
7,5,2022-02-01,Tuesday,Feb,2022,01
...,...,...,...,...,...,...
4879,556,2020-10-22,Thursday,Oct,2020,22
4885,557,2020-07-04,Saturday,Jul,2020,04
4943,558,2020-09-11,Friday,Sep,2020,11
4948,559,2020-10-11,Sunday,Oct,2020,11


# Transforming Product Table

In [32]:
product_dataset =pd.DataFrame(dataset['product'])
product_dataset = product_dataset.drop_duplicates()


In [33]:
product_dataset['product'].nunique()

188

In [34]:
product_dataset.insert(0, 'product_id', range(1, 1 + len(product_dataset)))
product_dataset

,product_id,product
0,1,Aquaspeed Aquarium Heater
1,2,Airpump Connector
2,3,Co2 Diffuser Set
4,4,Spirulina Tablet 50pcs
7,5,Dragon Stone Black Seiryu Ryou Stone
...,...,...
4634,184,x5-m and x5 aquarium lights
4657,185,Aquaspeed submersible pump A600 6watts 600LPH
4775,186,aquaruim scissors stainless (curved&straight) ...
4926,187,1.3 Meter Aquarium Siphon with Valve Fish Tank...


# Sentiment Table

In [35]:
sentiment_dataset =pd.DataFrame(dataset['sentiment'])
sentiment_dataset = sentiment_dataset.drop_duplicates()
sentiment_dataset.insert(0, 'sentiment_id', range(1, 1 + len(sentiment_dataset)))

sentiment_dataset



,sentiment_id,sentiment
0,1,Negative
416,2,Neutral
637,3,Positive


# Getting Word Frequency

In [36]:
word_dataset =pd.DataFrame(dataset[['sentiment','feedback']])


neutral= word_dataset['sentiment'].value_counts()['Neutral']
negative= word_dataset['sentiment'].value_counts()['Negative']
positive= word_dataset['sentiment'].value_counts()['Positive']

In [37]:
def generate_N_grams(text,ngram=2):
    words=[word for word in text.split(" ")]
    print(words)
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

In [38]:
positiveValues2=defaultdict(int)
negativeValues2=defaultdict(int)
neutralValues2=defaultdict(int)


In [39]:
for text in word_dataset[word_dataset.sentiment=='Positive'].feedback:
    for word in generate_N_grams(text,2):
        positiveValues2[word]+=1



['ok', 'sana', 'tumagal', 'sakin', 'incomplete', 'wala', 'sponge', 'back', 'flow', 'speed', 'restrictive']
['natapon', 'konti']
['small', 'size', 'only', 'inches', 'job']
['good', 'quality', 'well', 'packaged']
['good', 'product', 'overall', 'butas', 'lang', 'nung', 'dumating']
['goods']
['im', 'little', 'disappointed', 'kasi', 'late', 'dumating', 'parcel', 'know', 'problema', 'courier', 'think', 'seller', 'concern', 'wala', 'pala', 'parang', 'lagayan', 'small', 'pot', 'akala', 'kasama', 'hahaha', 'gutay', 'yellow', 'yellow', 'dahon']
['good', 'quality', 'accommodating', 'seller', 'well', 'packaged', 'order', 'again']
['okay', 'kaso', 'sumabog', 'yun', 'plastic', 'dating', 'sakin', 'tumapon', 'bawas', 'kalahating', 'kilo', 'yun', 'sana', 'pasecure', 'balot', 'next', 'time', 'lalo', 'matutulis', 'yun', 'bato', 'kasama', 'thank']
['maganda', 'sana', 'inayos', 'pack', 'halaman', 'stone', 'nasira', 'lalagyan', 'nung', 'halaman', 'weight', 'bato', 'buti', 'hindi', 'nasira', 'halaman']
['par

In [40]:
df_positive2=pd.DataFrame(sorted(positiveValues2.items(),key=lambda x:x[1],reverse=True), columns=['word','word_count'])
df_positive2 = df_positive2.head(10)
df_positive2['sentiment_id']=3
df_positive2

,word,word_count,sentiment_id
0,excellent quality,756,3
1,well packaged,684,3
2,order again,628,3
3,item shipped,499,3
4,shipped immediately,490,3
5,packaged item,417,3
6,accommodating seller,415,3
7,quality accommodating,373,3
8,immediately order,325,3
9,seller well,324,3


In [41]:
for text in word_dataset[word_dataset.sentiment=='Negative'].feedback:
    for word in generate_N_grams(text,2):
        negativeValues2[word]+=1

['deffective', 'item', 'seller', 'not', 'professional', 'wrong', 'move', 'dami', 'palang', 'bad', 'reviews', 'shop', 'nato', 'diko', 'nabasa', 'lesson', 'learned', 'review', 'muna', 'shop', 'item', 'buyyout']
['walang', 'dumating', 'gantong', 'item']
['damaged', 'defective', 'item']
['wlang', 'nalabas', 'co', 'buti', 'diy', 'diffuser']
['basag', 'poyung', 'dumating', 'aquaspeed', 'order', 'ibang', 'brand', 'dumating']
['pangit', 'nung', 'parts', 'nadumating', 'sira']
['kulang', 'dinidikit']
['tagal', 'ship', 'dumating']
['ordered', 'pcs', 'dumating', 'chatted', 'seller', 'yesterday', 'send', 'stone', 'crack', 'immedietly', 'held', 'see', 'vid', 'seller', 'doesn', 'response', 'yet', 'incident', 'got', 'dissapointed']
['scam', 'inorder', 'inch', 'quantity', 'dumating', 'sakin', 'inch', 'lang']
['uses', 'ayaw', 'gumana']
['bat', 'powder', 'binigay']
['not', 'receive', 'item']
['damaged', 'defective', 'item']
['disappointed', 'hindi', 'dumating', 'white', 'sand', 'worth', 'only', 'received

In [42]:
df_negative2=pd.DataFrame(sorted(negativeValues2.items(),key=lambda x:x[1],reverse=True), columns=['word','word_count'])
df_negative2 = df_negative2.head(10)
df_negative2['sentiment_id'] = 1
df_negative2

,word,word_count,sentiment_id
0,damaged defective,52,1
1,defective item,52,1
2,order again,31,1
3,not order,30,1
4,item shipped,28,1
5,shipped late,28,1
6,seller not,27,1
7,not responsive,20,1
8,poor quality,13,1
9,poor packaging,9,1


In [43]:
for text in word_dataset[word_dataset.sentiment=='Neutral'].feedback:
    for word in generate_N_grams(text,2):
        neutralValues2[word]+=1

['seller', 'freebies', 'wala', 'pala', 'sad', 'di', 'nagresponse', 'nung', 'place', 'order']
['ok', 'sana', 'lang', 'antagal', 'delivery', 'natuyo', 'dahon', 'sayang']
['late', 'delivery']
['matagal', 'lang', 'delivery']
['ok', 'quality', 'standard', 'packaging']
['item', 'shipped', 'time']
['item', 'damaged']
['ok', 'quality', 'standard', 'packaging']
['item', 'shipped', 'time']
['tumagas']
['shipping', 'delivery', 'late', 'items', 'well', 'packed', 'received', 'without', 'damage', 'anything', 'not', 'satisfied', 'never', 'tried', 'yet', 'fish', 'died', 'order', 'shipped']
['shipping', 'delivery', 'late', 'items', 'well', 'packed', 'received', 'without', 'damage', 'anything', 'not', 'satisfied', 'never', 'tried', 'yet', 'fish', 'died', 'order', 'shipped']
['ok', 'quality']
['maraming', 'maraming', 'salamat', 'seller', 'nareceive', 'inorder', 'submersible', 'led', 'lights', 'walang', 'damage', 'dumating', 'sobrang', 'ganda', 'kasi', 'pagkabalot', 'safe', 'safe', 'item', 'mali', 'pala',

In [44]:
df_neutral2=pd.DataFrame(sorted(neutralValues2.items(),key=lambda x:x[1],reverse=True), columns=['word','word_count'])
df_neutral2 = df_neutral2.head(10)
df_neutral2['sentiment_id']=2
df_neutral2

,word,word_count,sentiment_id
0,ok quality,67,2
1,standard packaging,28,2
2,item shipped,27,2
3,shipped time,27,2
4,quality standard,17,2
5,responsive seller,12,2
6,complete order,10,2
7,tagal dumating,9,2
8,quality responsive,7,2
9,order lang,6,2


In [45]:
bigram_dataset = df_positive2.append(df_negative2)
bigram_dataset = bigram_dataset.append(df_neutral2)



bigram_dataset

C:\Users\R\AppData\Local\Temp\ipykernel_12384\4226469948.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigram_dataset = df_positive2.append(df_negative2)
C:\Users\R\AppData\Local\Temp\ipykernel_12384\4226469948.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigram_dataset = bigram_dataset.append(df_neutral2)


,word,word_count,sentiment_id
0,excellent quality,756,3
1,well packaged,684,3
2,order again,628,3
3,item shipped,499,3
4,shipped immediately,490,3
5,packaged item,417,3
6,accommodating seller,415,3
7,quality accommodating,373,3
8,immediately order,325,3
9,seller well,324,3


In [46]:
bigram_dataset.insert(0, 'bigram_id', range(1, 1 + len(bigram_dataset)))
bigram_dataset


,bigram_id,word,word_count,sentiment_id
0,1,excellent quality,756,3
1,2,well packaged,684,3
2,3,order again,628,3
3,4,item shipped,499,3
4,5,shipped immediately,490,3
5,6,packaged item,417,3
6,7,accommodating seller,415,3
7,8,quality accommodating,373,3
8,9,immediately order,325,3
9,10,seller well,324,3


In [47]:
dataset = pd.merge(dataset, product_dataset, on='product', how='left')
del dataset['product']

In [48]:
dataset = pd.merge(dataset, sentiment_dataset, on='sentiment', how='left')
del dataset['sentiment']

In [49]:

dataset['date']=pd.to_datetime(dataset['date'])

dataset = pd.merge(dataset, date_dataset, on='date', how='left')


In [50]:
dataset=dataset.reindex(columns=['feedback_id','date_id','product_id','sentiment_id','rating','feedback'])
dataset

,feedback_id,date_id,product_id,sentiment_id,rating,feedback
0,1,1,1,1,1,deffective item seller not professional wrong ...
1,2,2,2,1,1,walang dumating gantong item
2,3,3,3,1,1,damaged defective item
3,4,3,4,1,1,wlang nalabas co buti diy diffuser
4,5,4,1,1,1,basag poyung dumating aquaspeed order ibang br...
...,...,...,...,...,...,...
4056,4057,264,180,3,5,goods item well packed mabilis ship dumating a...
4057,4058,179,180,3,5,second time ordering shop packed well items no...
4058,4059,21,180,3,5,excellent quality
4059,4060,27,180,3,5,excellent quality


In [51]:
with pd.ExcelWriter("fact_table.xlsx") as fact:
    dataset.to_excel(fact, sheet_name='Fact', index = False)
    date_dataset.to_excel(fact,sheet_name='Date',index=False)
    product_dataset.to_excel(fact,sheet_name='Product',index=False)
    sentiment_dataset.to_excel(fact,sheet_name='Sentiment',index=False)
    bigram_dataset.to_excel(fact,sheet_name='Bigram',index=False)
    
    